<a href="https://colab.research.google.com/github/nom682025-cell/Crazi-Di/blob/main/examples/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B1%D0%B5%D0%B7%20%D1%83%D1%87%D0%B8%D1%82%D0%B5%D0%BB%D1%8F%20(%D0%BA%D0%BB%D0%B0%D1%81%D1%82%D0%B5%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F%2C%20PCA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#импортируем библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_decision_regions

# Загрузка данных

Загружаем датасет, подготовленный на предыдущих этапах обучения.

In [ ]:
import json, requests, urllib, io
import pandas
import os

from google.colab import userdata
pao = userdata.get('GITHUB_TOKEN_BIOMED_UPSKILL')
user='AI-is-out-there'

github_session = requests.Session()
github_session.auth = (user, pao)

# providing raw url to download csv from github
csv_url = 'https://raw.githubusercontent.com/AI-is-out-there/biomed-upskill-dev/main/datasets/heart3.csv'

download = github_session.get(csv_url).content
raw_table_data = pandas.read_csv(io.StringIO(download.decode('utf-8')), header = 0)
raw_table_data.head(3)

In [ ]:
#смотрим размерность датасета
raw_table_data.shape

# Helicopter view

In [ ]:
#смотим данные датасета в табличном виде - первые пять строк
table_data = raw_table_data
table_data.head()

In [ ]:
#статистические параметры распределения по столбцам
table_data.describe()

In [ ]:
#проверяем распределение целевого признака
sns.countplot(data=table_data, x="ССЗ")
plt.title("Распределение целевой переменной 'ССЗ'")
plt.show()

# Визуализация распределения значений датасета

In [ ]:
n = table_data.shape[1]  # Количество столбцов
cols_per_row = 4  # Сколько графиков в одной строке
n_rows = (n + cols_per_row - 1) // cols_per_row  # Вычисляем количество строк

fig, axes = plt.subplots(
    n_rows,
    cols_per_row,
    figsize=(cols_per_row * 4, n_rows * 3),  # Размер фигуры (ширина, высота)
    squeeze=False  # Гарантирует, что axes всегда будет двумерным массивом
)

# Убираем лишние оси, если столбцов не кратно cols_per_row
for i in range(n, n_rows * cols_per_row):
    fig.delaxes(axes.flatten()[i])

# Строим боксплоты
for i, col in enumerate(table_data.columns):
    row_idx = i // cols_per_row
    col_idx = i % cols_per_row
    ax = axes[row_idx, col_idx]

    sns.boxplot(
        y=table_data.iloc[:, i],
        data=table_data,
        ax=ax,
        medianprops={"color": "r", "linewidth": 2}
    )
    ax.set_title(col, fontsize=10)  # Подпись графика

plt.tight_layout()  # Автоматическая настройка отступов
plt.show()

In [ ]:
#проверяем корреляцию признаков

sns.heatmap(table_data.corr(),annot=True,fmt="0.2f",cmap="coolwarm")
plt.show()

In [ ]:
#изобразим только те значения, которые имеют значительную корреляцию
df_corr_table = table_data.corr()
mask_con_corr = df_corr_table[(df_corr_table >= 0.50) | (df_corr_table <= -0.50)]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(15,15))
fig = sns.heatmap(mask_con_corr, vmin=-1, vmax=1, annot=True, fmt='0.2f')
fig.set_title("Тепловая карта корреляции (уровень 1/2)")

# Методы PCA - princopal component analysis

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
table_data_pca = table_data
table_data_pca = table_data_pca.drop('ССЗ', axis = 1)
table_data_pca.columns

In [ ]:
# Before applying PCA, each feature should be centered (zero mean) and with unit variance
scaled_data = StandardScaler().fit(table_data_pca).transform(table_data_pca)

#pca = PCA(n_components = 2).fit(scaled_data) #количество компонет

pca = PCA(n_components = 0.7).fit(scaled_data) #значение дисперсии, которое определит необходимое количество компонент

x_pca = pca.transform(scaled_data)
print(table_data.shape, x_pca.shape)

In [ ]:
percent = pca.explained_variance_ratio_
print(percent)
print(sum(percent))
#To see how much variance is preserved for each dataset.

In [ ]:
def pca_explained(X, threshold):
  features = X.shape[1]
  for i in range(2, features):
    pca = PCA(n_components = i).fit(X)
    sum_ = pca.explained_variance_ratio_
    # add all components explained variances
    percent = sum(sum_)
    print('{} components at {:.2f}% explained variance'.format(i,percent*100))
    if percent > threshold:
      break

pca_explained(scaled_data, 0.85)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x_pca[:,0], x_pca[:,1], c=table_data['ССЗ'], cmap='plasma', alpha=0.4, edgecolors='black', s=65);
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')

# ICA independed component analysis

In [ ]:
from sklearn.decomposition import FastICA

ICA = FastICA(n_components=2)
scaled_data_ica = ICA.fit_transform(scaled_data)


plt.figure(figsize=(8,6))
plt.scatter(scaled_data_ica[:,0],
            scaled_data_ica[:,1],
            c=table_data['ССЗ'],
            cmap='plasma',
            alpha=0.4,
            edgecolors='black',
            s=65
            );
plt.xlabel('First Independed Component')
plt.ylabel('Second Independed Component')

# Кластеризация

In [ ]:
from sklearn.cluster import KMeans

pca_data = pd.DataFrame(x_pca[:,0:2],columns=['PC1','PC2'])


kmeans = KMeans(n_clusters=4).fit(scaled_data)
pca_data['cluster'] = pd.Categorical(kmeans.labels_)
sns.scatterplot(x="PC1",y="PC2",hue="cluster",data=pca_data)

In [ ]:
scaled_data.shape[1]

In [ ]:
wcss=[]
#WCSS (Within-Cluster Sum of Squares) is a metric used to evaluate the compactness of clusters.
num_cluster = scaled_data.shape[1]
for i in range(1,num_cluster):
    kmeans=KMeans(n_clusters=i, init='k-means++', random_state=37)
    kmeans.fit(scaled_data)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10,8))
plt.plot(range(1,num_cluster), wcss)
plt.xlabel("Number of Clusters")
plt.ylabel("WCSS")
plt.title("K-Means Clustering")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4).fit(scaled_data)

df_segm_kmeans=table_data_pca.copy()
df_segm_kmeans['Segment k-Means']=kmeans.labels_
df_segm_analysis=df_segm_kmeans.groupby(['Segment k-Means']).mean()
df_segm_analysis


In [ ]:
df_segm_kmeans

In [ ]:
df_segm_analysis['N-obs']=df_segm_kmeans[['ЛПВП','Segment k-Means']].groupby(['Segment k-Means']).count()
df_segm_analysis['Prop-obs']=df_segm_analysis['N-obs']/df_segm_analysis['N-obs'].sum()

x_axis=df_segm_kmeans['Холестерин']
y_axis=df_segm_kmeans['Индекс массы тела, кг/м^2']
plt.figure(figsize=(12,9))
colors = ['r', 'g', 'b', 'y']
sns.scatterplot(x=x_axis,y=y_axis, hue=df_segm_kmeans['Segment k-Means'], palette=colors)
plt.title("Segmented K-Means")
plt.show()

In [ ]:
# put feature values into dataframe

plt.figure(figsize=(10,8))
# main scatterplot
plt.scatter(x_pca[:,0], x_pca[:,1], c=table_data['ССЗ'],
            cmap='plasma', alpha=0.4, edgecolors='black', s=40);
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')


In [ ]:
fig = plt.figure(figsize=(8, 4))
plt.imshow(pca.components_, interpolation = 'none', cmap = 'plasma')
feature_names = list(table_data_pca.columns)
plt.gca().set_xticks(np.arange(-.5, len(feature_names)-1));
plt.gca().set_yticks(np.arange(0.5, 2));
plt.gca().set_xticklabels(feature_names, rotation=90, ha='left',fontsize=12);
plt.gca().set_yticklabels(['First PC', 'Second PC'], va='bottom',fontsize=12);
plt.colorbar(orientation='horizontal', ticks=[pca.components_.min(), 0,
                                              pca.components_.max()],pad=0.65);

# MDS t-SNE

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS

In [ ]:
mds = MDS(n_components = 2)
scaled_data_mds = mds.fit_transform(scaled_data)

plt.scatter(scaled_data_mds[:,0], scaled_data_mds[:,1], c=table_data['ССЗ'], cmap='plasma', alpha=0.4, edgecolors='black', s=65);
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=0, perplexity=10)
scaled_data_mds_tsne = tsne.fit_transform(scaled_data)
plt.scatter(scaled_data_mds_tsne[:,0], scaled_data_mds_tsne[:,1], c=table_data['ССЗ'], cmap='plasma', alpha=0.4, edgecolors='black', s=65);
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import ward, dendrogram, fcluster

In [ ]:
#scaled_data, data_table['Healthy_status'] = make_blobs(random_state = 10)
#n_clusters must be None if distance_threshold is not None
cls = AgglomerativeClustering(n_clusters = 3,linkage='ward', distance_threshold=None)
cls_assignment = cls.fit_predict(scaled_data)

In [ ]:
Z = ward(scaled_data)
plt.figure(figsize=(17,5));
R = dendrogram(Z, truncate_mode='lastp', p=7, orientation='top', leaf_font_size=8)
plt.title('Hierarchical Clustering Dendrogram (truncated and grouped)')
plt.xlabel('sample index')
plt.ylabel('distance')
plt.show()

# Affinity propagation

In [ ]:
from sklearn import metrics
from sklearn.cluster import AffinityPropagation
from sklearn.metrics import adjusted_rand_score

In [ ]:
def plot_connected_points(X, labels, centers, cmap):
    for i in range(len(X)):
        color = cmap(labels[i] / len(centers))
        plt.plot([X[i, 0], centers[labels[i], 0]], [X[i, 1], centers[labels[i], 1]], c=color, alpha=0.8)

In [ ]:
sk_ap = AffinityPropagation()
sk_ap_pred_res = sk_ap.fit_predict(scaled_data)
sk_ap_ari = adjusted_rand_score(table_data['ССЗ'], sk_ap_pred_res)
print(f'Adjusted Rand Score for sk AffinityPropagation: {sk_ap_ari}', '', sep='\n')
print('Number of clusters = ', np.max(sk_ap_pred_res))
#print('prediction', sk_ap_pred_res, sep='\n')

In [ ]:
plt.figure(figsize=(12, 5))

plt.scatter(scaled_data[:, 0], scaled_data[:, 1], c=sk_ap_pred_res, cmap='rainbow', s=10)
plt.scatter(sk_ap.cluster_centers_[:, 0], sk_ap.cluster_centers_[:, 1], c='black', s=50)
plt.title('AffinityPropagation (scikit-learn)')
plt.xlabel(table_data.columns[5])
plt.ylabel(table_data.columns[29])

plot_connected_points(scaled_data, sk_ap_pred_res, sk_ap.cluster_centers_, plt.cm.rainbow)

plt.show()

# Agglomerative Clustering

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram

from sklearn.cluster import AgglomerativeClustering

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    #plt.figure(figsize=(17,5));
    dendrogram(linkage_matrix, **kwargs)
    plt.rcParams["figure.figsize"] = (17,5)
    plt.xlabel("Number of points in node (or index of point if no parenthesis).")
    plt.show()



In [ ]:
# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

model = model.fit(scaled_data)
plt.title("Hierarchical Clustering Dendrogram")
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode="level", p=3)
#

# дополнительная секция - расширенная программа со специальными приемами машинного обучения

# ML-классификация: модель GaussianNaiveBayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split


#GaussianNaiveBayes

In [ ]:
# set the classification index of table
clf_index = 33

# classification variable
y1 = table_data.iloc[:, clf_index]
# input variable
X1 = table_data.drop(table_data.columns[[clf_index]], axis=1).iloc[:,:]

y1 = pd.Series(LabelEncoder().fit_transform(y1))
X1_train, X1_test, y1_train, y1_test = train_test_split(X1.values,
                                                        y1.values,
                                                        test_size=0.3,
                                                        random_state=37,
                                                        stratify=y1.values)

In [ ]:

def decision_boundary_plot(X, y, X_train, y_train, clf, feature_indexes, title=None):
    feature1_name, feature2_name = X.columns[feature_indexes]
    X_feature_columns = X.values[:, feature_indexes]
    X_train_feature_columns = X_train[:, feature_indexes]
    clf.fit(X_train_feature_columns, y_train)

    plt.figure(figsize=(8,6))
    plot_decision_regions(X=X_feature_columns, y=y.values, clf=clf)
    plt.xlabel(feature1_name)
    plt.ylabel(feature2_name)
    plt.title(title)

In [ ]:
table_data.columns

In [ ]:
sk_nb_clf = GaussianNB()
sk_nb_clf.fit(X1_train, y1_train)
sk_nb_clf_pred_res = sk_nb_clf.predict(X1_test)
sk_nb_clf_accuracy = accuracy_score(y1_test, sk_nb_clf_pred_res)

print(f'sk Naive Bayes classifier accucacy: {sk_nb_clf_accuracy}')
print(sk_nb_clf_pred_res)

feature_indexes = [5, 29]
title1 = 'GaussianNB surface'
decision_boundary_plot(X1, y1, X1_train, y1_train, sk_nb_clf, feature_indexes, title1)

# ML - Описание классов и матрица ошибок

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
table_data.columns

In [ ]:
# set the classification index of table
clf_index = 33

In [ ]:
plt.figure(figsize=(35,20),dpi=90)
# Plot frequency percentages barplot
table_data[table_data.columns[clf_index]].value_counts(normalize=True).mul(100).plot(kind='barh', width=0.8, figsize=(8,5))

# Add frequency percentages to the plot
labels = table_data[table_data.columns[clf_index]].value_counts(normalize=True).mul(100).round(1)
for i in labels.index:
    plt.text(labels[i], i, str(labels[i])+ '%', fontsize=15, weight='bold')

plt.xlim([0, 110])
plt.xlabel('Frequency Percentage', fontsize=13)
plt.ylabel(table_data.columns[clf_index], fontsize=13)
plt.title('Frequency Percentage of Target Classes', fontsize=13)
plt.show()

In [ ]:
# set the classification index of table
clf_index = 33

# classification variable
y1 = table_data.iloc[:, clf_index]
# input variable
X1 = table_data.drop(table_data.columns[[clf_index]], axis=1).iloc[:,:]

y1 = pd.Series(LabelEncoder().fit_transform(y1))
X1_train, X1_test, y1_train, y1_test = train_test_split(X1.values,
                                                        y1.values,
                                                        test_size=0.3,
                                                        random_state=0,
                                                        stratify=y1.values)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X1.values,
                                                 y1.values,
                                                 test_size=0.3,
                                                 random_state=41,
                                                 stratify=y1.values)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
GussianClassifier = GaussianNB()
GussianClassifier.fit(X1_train,y1_train)
y_pred=GussianClassifier.predict(X_test)
print("===================================> Result <===================================")
print("Accuracy                     = " ,metrics.accuracy_score(y_test,y_pred))
print("F1 Score                     = " ,metrics.f1_score(y_test,y_pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
target_names = ['True', 'False']
labels_names = [0,1]
print(classification_report(y_test, y_pred,labels=labels_names, target_names=target_names))
#cm = confusion_matrix(y_test, y_pred,labels=labels_names,normalize='true')
cm = confusion_matrix(y_test, y_pred,labels=labels_names)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=target_names)
disp = disp.plot(cmap=plt.cm.Blues,values_format='g')
plt.show()